<a href="https://colab.research.google.com/github/Juniorffonseca/python-valorant-preditor/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazendo meu projeto do TCC, que foi feito em R, na linguagem Python.
O projeto consiste em usar redes neurais para prever resultado de partidas do cenário profissional de Valorant


In [19]:
# Carregando pacotes
import re
import requests
from bs4 import BeautifulSoup
import itertools
import pandas as pd

In [2]:
# Criando variável páginas e criando variável 'p' que será a parte final do url (o número da página)
paginas = []
p = 1

# Criando um laço for que armazenará o url de cada página dentro da variável paginas
for i in range(1, 51):
    paginas.append(f'https://www.vlr.gg/matches/results/?page={p}')
    p += 1

# Variável partidas e variável c
partidas = []
c = 1

# Função que retorna o url de cada partida
def funcaoPagina(pagina):
    response = requests.get(pagina)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    partidas = [link.get('href') for link in links]
    partidas = partidas[14:64]
    partidas = ['https://www.vlr.gg' + partida for partida in partidas]
    return partidas


In [14]:
# Criando f e uma lista que receberá todos os returns da funcaoPagina (url de cada partida)
f = 0
a = []

# Executando um for que fará a iteração da funcaoPagina todas as vezes necessárias
for i in paginas:
    a.append(funcaoPagina(paginas[f]))
    f += 1

a = list(itertools.chain(*a))

In [27]:
def medias_Jogadores(url_jogador):
    html_lido = requests.get(url_jogador).text
    soup = BeautifulSoup(html_lido, 'html.parser')
    dados_jogador = []
    for table in soup.find_all('table'):
        dados_jogador.append(table.text)
    dados_jogador = [x.strip().split("\n") for x in dados_jogador]
    dados_jogador = [x for x in dados_jogador if x[0] in ['Use', 'Rating', 'ACS', 'KAST', 'K:D', 'ADR']]
    dados_jogador = [{dados_jogador[i][0]: float(dados_jogador[i][1])} for i in range(len(dados_jogador))]
    dados_jogador = {k: v * dados_jogador[0]['Use'] for d in dados_jogador for k, v in d.items()}
    dados_jogador = {k: round(v / dados_jogador['Use'], 2) for k, v in dados_jogador.items()}
    medias_jogador = dados_jogador
    #medias_jogador['KAST'] = round(medias_jogador['KAST'], 0)
    return medias_jogador


In [28]:
def get_Ganhadores(url_partida):
    html = requests.get(url_partida).text
    soup = BeautifulSoup(html, 'html.parser')
    spoilers = soup.find_all("div", class_="js-spoiler")
    spoilers = [spoiler.text.strip().replace("\t", "").replace("\n", "") for spoiler in spoilers]
    placar = "".join(spoilers)
    match = re.search("(\d+):(\d+)", placar)
    time1, time2 = int(match.group(1)), int(match.group(2))
    ganhador = 1 if time1 > time2 else 0
    return ganhador


In [34]:
def medias_Times(url_partida):
  try:
    # Pegando os dados no link da partida
    html = requests.get(url_partida).text
    soup = BeautifulSoup(html, 'html.parser')
    nodes = soup.select('td.mod-player a')
    links_jogadores = [node['href'] for node in nodes]

    # Separando os nomes dos jogadores de cada time em 2 arrays
    timeA = links_jogadores[:5]
    timeB = links_jogadores[5:]

    # Criando os links usando os nomes dos jogadores
    timeA = ['https://www.vlr.gg/?timespan=all' + i for i in timeA]
    timeB = ['https://www.vlr.gg/?timespan=all' + i for i in timeB]

    timeA_medias = []
    timeB_medias = []

    for i in timeA:
        timeA_medias.append(medias_Jogadores(i))

    for i in timeB:
        timeB_medias.append(medias_Jogadores(i))

    print(timeA_medias)
    print(timeB_medias)
    # timeA_medias = pd.concat(timeA_medias, axis=1)
    # timeB_medias = pd.concat(timeB_medias, axis=1)

    timeA_medias = timeA_medias.drop('Use', axis=1)
    timeB_medias = timeB_medias.drop('Use', axis=1)

    partida = pd.concat([timeA_medias, timeB_medias], axis=1)

    partida.columns = ['time1R', 'time1ACS', 'time1KAST', 'time1KD', 'time1ADR',
                       'time2R', 'time2ACS', 'time2KAST', 'time2KD', 'time2ADR']

    partida = partida[['time1R', 'time2R', 'time1ACS', 'time2ACS', 'time1KAST', 'time2KAST',
                      'time1KD', 'time2KD', 'time1ADR', 'time2ADR']]

    partida = partida.mean(axis=1).transpose()

    ganhador = get_Ganhadores(url_partida)

    partida['ganhador'] = ganhador

    return partida

  except Exception as e:
    print('error:', e)


In [35]:
medias_Times(a[1])

[{}, {}, {}, {}, {}]
[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
error: 'list' object has no attribute 'drop'
